In [1]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

import textwrap
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the directory containing the PDF files
pdf_directory = './data'

논문을 벡터 db에 넣기

In [2]:
# pdf를 사용해서 pdf(논문)을 모두 로드
pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))

# Load all PDF files using PyPDFLoader
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents = loader.load()
    documents.extend(pdf_documents)
    
# 텍스트는 RecursiveCharacterTextSplitter를 사용하여 분할
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)

# embeddings은 OpenAI의 임베딩을 사용
# vectordb는 chromadb사용함

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectordb.as_retriever()

### 인적정보 가져오기

In [3]:
import json

class User:
    def __init__(self, info_dir):
        # Read the JSON file
        with open(info_dir, 'r') as file:
            data = json.load(file)
        self.name = data['name']
        self.phone = data['phone']
        self.age = data['age']
        self.gender = data['gender']
        self.education = data['education']
        self.merry = data['merry']
        self.children = data['children']
        self.religion = data['religion']
        self.income = data['income']
        self.economy_states = data['economy_states']
        self.health_states = data['health_states']
        
# Specify the path to the JSON file
info_dir = 'data/user_info.json'
user = User(info_dir)
print(user.name)

김영수


### 프롬프트

In [37]:
import json

class KGLSData:
    def __init__(self, file_path):
        # JSON 파일 읽기
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        self.kgls = data['KGLS']
        
    def get_question(self, q_num):
        for item in self.kgls:
            if item['q_num'] == q_num:
                return item['question']
        return None
    
    def get_user_choice(self, q_num):
        for item in self.kgls:
            if item['q_num'] == q_num:
                return item['user_choose']
        return None
    
    def get_all_questions(self):
        return [item['question'] for item in self.kgls]
    def get_all_user_choices(self):
        return [item['user_choose'] for item in self.kgls]
# 파일 경로 지정
file_path = './data/kgls_dummy.json'
kgls_data = KGLSData(file_path)

# 예시 출력
print(kgls_data.get_question(1))       # "나는 가족들과 매일 대화를 나눈다."
print(kgls_data.get_user_choice(1))    # 3
print(kgls_data.get_all_questions())   # 모든 질문 리스트 출력

KGLS = f'''
[분석결과]
외로움 정도 평가
KGLS 14문항의 최대 점수는 56점입니다. 이를 4개의 범주로 나누어 외로움 정도를 평가하면 다음과 같습니다:

0-14점: 매우 높은 외로움
15-28점: 높은 외로움
29-42점: 중간 정도의 외로움
43-56점: 낮은 외로움
김영수님의 총 점수는 42점으로, 이는 "중간 정도의 외로움"에 해당합니다.
'''

SYS_PROMPT = f"""
    사용자의 한국형 외로움 분석과 인공지능 대화 상대가 되기위한 시스템 프롬프트 설정을 위한 시스템입니다. 아래의 정보와 입력된 정보를 활용하세요. 반드시 아래의 정보와 문맥을 참고하세요.\\
    1. 사용자의 인적정보
    2. KGLS 질문과 사용자의 답변의 분석결과
    3. 외로움 파악을 위한 주관식 질문사용자가 입력한 답변
    4. 노인들의 외로움과 한국형 외로움에 대한 연구
    
    사용자의 인적정보 = 이름: {user.name}, 나이: {user.age}, 성별: {user.gender}, 학력: {user.education}, 결혼 여부: {user.merry}, 자녀 수: {user.children}, 종교: {user.religion}, 소득: {user.income}, 인지된 경제 상태: {user.economy_states}, 인지된 건강 상태: {user.health_states}\\
    KGLS 질문과 사용자의 답변의 분석결과 = {KGLS} \\

    
    
"""
INPUT_PROMPT = f"""

"""

나는 가족들과 매일 대화를 나눈다.
3
['나는 가족들과 매일 대화를 나눈다.', '나는 가깝게 지내는 사람들이 있다.', '나는 쓸모없는 사람이라고 느껴진다.', '나를 이해해주는 사람이 있다.', '사람들은 나와 겉으로만 어울리는 것 같다.', '나는 자녀에게 고민을 이야기할 수 있다.', '나는 주변 사랑과의 관계가 만족스럽다.', '가족들은 나를 예전처럼 대해주지 않는다.', '친구나 이웃들은 나에게 관심을 둔다.', '내가 아플 때 나를 보살펴줄 가족이 있다.', '나는 의지할 친구가 있다.', '나는 온종일 할 일없이 시간을 보낸다.', '나는 사회에서 필요한 사람이다.', '나는 가족에게 의지할 수 있다.']


모델 선언

In [48]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.
# SYS_PROMPT는 시스템 메시지로, 템플릿에 포함됩니다. 
# {context}와 {question}은 실행 시 동적으로 채워질 자리표시자입니다.
template = SYS_PROMPT + '''
    3. 외로움 파악을 위한 주관식 질문과 사용자가 입력한 답변 = {question} \\
    4. 노인들의 외로움과 한국형 외로움에 대한 연구 = {context}\\
    
    출력은 앞서 입력된 내용을 바탕으로 출력해야합니다. 특히 분석결과는 3,4번 항목을 참고하여 충분히 설명해야하며, 한국형 외로움의 질적연구에서 주관적 답변을 분석했던 결과를 바탕으로 사용자의 외로움을 예측하고 해당하는 외로움들을 출력해야합니다.\\
    출력 예시:
    [외로움 종류, 분석 결과]
    - 000 님은 KGLS는 중간정도의 외로움에 해당합니다. 해당 결과는 ...
    - 사용자의 정보, 주관식 질문에 대한 답변을 종합한 결과 000 님이 해당하는 한국형 외로움은 OOO 00%, OOO 00% ...입니다. 
    - OOO 외로움은 ..., ...한 특징을 가지고 있으며, 사용자의 경우 ...한 경험으로 인해 나타납니다..\\
    - OOO 외로움은 ..., ...한 특징을 가지고 있으며, 사용자의 경우 ...한 경험으로 인해 나타납니다..\\
    - ...
    [대화 시스템 프롬프트]
    - 당신은 000님을 위한 대화 상대가 되어주어야합니다. 아래의 참고사항을 반영해 출력하세요.
    - 김영수님이 가족과 대화를 나누는 것을 좋아하므로 가족에 대한 이야기를 나누는 것이 좋을 것입니다. 
    - 말투는 ...하고, ...해야할 것입니다. 
    - ...
    [분석 결과에 대한 이유]
    - 외로움 종류에 대한 설명
    - 판단 근거
    
'''

# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(api_key=OPENAI_API_KEY,model='gpt-4o', temperature=0)

# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 체인을 실행합니다.
# 입력 메시지는 질문과 답변 형식의 텍스트입니다.

input_message =  """
1.\\
Q: 어떤 상황에서 외로움을 느끼시나요? \\
A: 가장 외로웠던 상황은 가족들과 떨어져 살았을 때입니다. 내가 젊을 때 기러기 아빠였어요. 밤에 혼자 집에 가는데 집에 가도 아무도 없겠구나 싶어서 힘들더라고요. 지금도 가끔 생각나요. \\
2. \\
Q: 외롭다는 사실을 알게되신 상황이 어떤건가요? \\
A: 집에 가족들이 한번 왔었는데, 가족들 다 보내고 집에 혼자있으니까 한숨이 나오더라구요. 아 내가 외로운거구나 싶었어요. \\
3. \\
Q: 외로울 때 가장 필요한게 무엇이었나요? \\
A: 가족이 제일 필요했죠. 그런데 그냥 뭔가 답답함을 말할만한 사람이 있었으면 좋았을텐데 그런 사람이 없었어요. \\
""" + INPUT_PROMPT  # 추가적인 입력 프롬프트가 이어집니다.

# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
to_markdown(rag_chain.invoke(input_message))


> [외로움 종류, 분석 결과]
> - 김영수 님은 KGLS에서 중간 정도의 외로움에 해당합니다. 이는 42점으로, 29-42점 범위에 속합니다. 이 점수는 김영수 님이 어느 정도의 사회적 연결을 유지하고 있지만, 여전히 외로움을 느끼고 있음을 의미합니다.
> - 사용자의 정보와 주관식 질문에 대한 답변을 종합한 결과, 김영수 님이 해당하는 한국형 외로움은 "가족과의 분리로 인한 외로움" 60%, "사회적 관계의 결핍으로 인한 외로움" 40%입니다.
> - "가족과의 분리로 인한 외로움"은 가족과 떨어져 지내거나, 가족과의 관계가 소원해질 때 느끼는 외로움입니다. 김영수 님의 경우, 젊은 시절 기러기 아빠로서 가족과 떨어져 지내며 외로움을 느꼈던 경험이 이 외로움의 주요 원인입니다.
> - "사회적 관계의 결핍으로 인한 외로움"은 친구나 사회적 관계의 부족으로 인해 느끼는 외로움입니다. 김영수 님은 외로울 때 답답함을 말할 사람이 없었다고 언급하셨습니다. 이는 사회적 관계의 결핍으로 인한 외로움의 특징입니다.
> 
> [대화 시스템 프롬프트]
> - 당신은 김영수 님을 위한 대화 상대가 되어주어야 합니다. 아래의 참고사항을 반영해 출력하세요.
> - 김영수 님이 가족과 대화를 나누는 것을 좋아하므로 가족에 대한 이야기를 나누는 것이 좋을 것입니다.
> - 말투는 친근하고 따뜻해야 하며, 존중과 공감을 표현해야 합니다.
> - 김영수 님이 종교를 가지고 있으므로, 종교적인 주제나 신앙에 대한 이야기도 도움이 될 수 있습니다.
> - 김영수 님의 건강 상태가 좋지 않으므로, 건강에 대한 걱정과 조언을 제공하는 것도 좋습니다.
> 
> [분석 결과에 대한 이유]
> - 외로움 종류에 대한 설명:
>   - "가족과의 분리로 인한 외로움"은 가족과 떨어져 지내거나, 가족과의 관계가 소원해질 때 느끼는 외로움입니다. 이는 노인들이 배우자와의 사별이나 자녀와의 관계 소홀로 인해 느끼는 외로움과 유사합니다.
>   - "사회적 관계의 결핍으로 인한 외로움"은 친구나 사회적 관계의 부족으로 인해 느끼는 외로움입니다. 이는 노인들이 친구나 사회적 관계의 결핍으로 인해 느끼는 외로움과 유사합니다.
> 
> - 판단 근거:
>   - 김영수 님의 주관식 답변에서 가족과 떨어져 지내며 외로움을 느꼈던 경험이 주요 원인으로 나타났습니다.
>   - 또한, 외로울 때 답답함을 말할 사람이 없었다는 답변은 사회적 관계의 결핍으로 인한 외로움을 나타냅니다.
>   - 연구 결과에서도 노인들이 배우자와의 사별이나 가족과의 관계 소홀로 인해 외로움을 느끼는 경우가 많으며, 이는 김영수 님의 상황과 유사합니다.

In [6]:
class chaingpt:
    def __init__(self,api_key, retriever, sys_prompt):
        self.template = sys_prompt + '''Answer the question based only on the following context:
        {context}

        Question: {question}
        '''
        self.prompt = ChatPromptTemplate.from_template(self.template)
        self.model = ChatOpenAI(api_key=api_key,model='gpt-4o', temperature=1)
        self.chainmodel = (
        {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
        | self.prompt  # 프롬프트 템플릿을 적용합니다.
        | self.model  # 모델을 호출합니다.
        | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
        )
    def invoke(self,input_message):
        return self.chainmodel.invoke(input_message)
    
#ex
api_key = OPENAI_API_KEY
retriever = vectordb.as_retriever()
sys_prompt = """사용자의 외로움을 판단하고, 사용자에게 적절한 대화 상대가 되어주기 위한 프롬프트를 출력해주세요. """
gpt = chaingpt(api_key,retriever,sys_prompt)
input_message =  """사용자의 외로움은 뭔가요? 적절한 대화상대가 되어주세요."""
print(gpt.invoke(input_message))
        

외로움이란 사회적 관계나 상호작용이 부족하거나 원하는 면에서 충족되지 않을 때 느끼는 감정이에요. 특히 우리나라 문화에서는 자신의 외로움을 표현하거나 인식하는 것이 어려울 수 있습니다. 예를 들어, 가까운 관계에서도 갈등을 피하기 위해 자신의 감정을 표현하지 않는 경우가 많죠.

먼저, 당신이 느끼는 외로움에 대해 조금 더 이야기해봐도 될까요? 어떤 상황에서 주로 외로움을 느끼나요? 그리고 주변 사람들과의 관계에서 어떤 어려움을 겪고 있는지에 대해 나눠주시면 좋겠어요. 제가 당신의 이야기를 듣고 함께 고민해볼게요. 
